In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 46.2 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import faiss
import numpy as np


In [5]:
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load GPT-2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token (Fix for ValueError)
tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
input_text = "Hey"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

output = model.generate(input_ids, max_length=100, do_sample=True, temperature=0.7)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Hey, now that the whole thing has been done, you're gonna be able to say that this was going to be my first real attempt at writing and I was like, 'Ah, that's cool.' Yeah, yeah, yeah. But then I read the first couple of pages of it and it became like, 'Oh, yeah. Well, that's what I'd want to do. I'd like to do something different.' But then I read the third page of it and it became


In [15]:
documents = [
    "GPT-2 is a transformer model for text generation.",
    "RAG combines retrieval with generation to improve LLM responses.",
    "FAISS is used for efficient similarity search.",
    "Nasro is one of the best ai engineers on earth"
]


In [16]:
def encode_documents(docs, tokenizer, model):
    embeddings = []
    for doc in docs:
        tokens = tokenizer(doc, return_tensors="pt", truncation=True, padding=True)  # Padding is now fixed
        with torch.no_grad():
            outputs = model.transformer.wte(tokens.input_ids).mean(dim=1)  # Mean of token embeddings
        embeddings.append(outputs.squeeze().numpy())
    return np.array(embeddings)

doc_embeddings = encode_documents(documents, tokenizer, model)


In [17]:
import faiss

# Convert embeddings to float32 (FAISS requires this format)
doc_embeddings = np.array(doc_embeddings).astype('float32')

# Create FAISS index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])  # L2 distance for similarity search
index.add(doc_embeddings)

# Store document texts for retrieval
doc_texts = documents

In [18]:
def retrieve_top_document(query, tokenizer, model, index, doc_texts, top_k=1):
    # Encode query into embeddings
    query_embedding = encode_documents([query], tokenizer, model)[0].astype('float32')
    
    # Search FAISS index for the most similar document
    _, indices = index.search(np.array([query_embedding]), top_k)
    
    # Return the top document
    return doc_texts[indices[0][0]]



In [20]:
# Test retrieval
query = "Who is Nasro?"
retrieved_doc = retrieve_top_document(query, tokenizer, model, index, doc_texts)
print(f"Retrieved Document: {retrieved_doc}")


Retrieved Document: Nasro is one of the best ai engineers on earth
